**Installation of SimPy""
If not available**

In [66]:
# Window
!python -m pip install simpy
!python -m pip install numpy
!python -m pip install matplotlib


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


**Import of required modules**

In [67]:
import simpy
import numpy as np
import numpy.random as random

**Parameters settings**

In [68]:
MAXSIMTIME              = 100000
VERBOSE                 = False
LAMBDA                  = 3.8
MU                      = 16.0
POPULATION              = 50000000
SERVICE_DISCIPLINE      = 'FIFO'
LOGGED                  = False
PLOTTED                 = False

**Discrete-Event Simulation model**


**The definition of a job**.

The properties of a job are


1. job execution time
2. job arrival time

In [69]:
class Job:
    def __init__(self, name, arrtime, duration):
        self.name = name
        self.arrtime = arrtime
        self.duration = duration

    def __str__(self):
        return '%s at %d, length %d' %(self.name, self.arrtime, self.duration)

**Disciplines**
Different queue disciplines can be defined here.


1.   Shortest Job First (SJF)
2.   List item



In [70]:
def SJF( job ):
    return job.duration

**The definition of server**

 There are 3 arguments needed for a server:
 1. env: SimPy environment
 2. queue discipline: 
   - FIFO: First In First Out
   - SJF : Shortest Job  
 3. name: name of sever 1 or 2

In [71]:
class Server:
    def __init__(self, env, name, nextServer, strat = 'FIFO'):
        self.name = name
        self.nextServer = nextServer
        
        self.env = env
        self.strat = strat
        self.Jobs = list(())
              
        self.serversleeping = None
        
        ''' statistics '''
        self.waitingTime = 0
        self.idleTime = 0
        self.jobsDone = 0

        ''' register a new server process '''
        env.process( self.serve() )

    def nJobs(self):
        return len(self.Jobs)

    def serve(self):
        while True:
            # do nothing, just change server to idle
            # and then yield a wait event which takes infinite time
            if len( self.Jobs ) == 0 :
                self.serversleeping = self.env.process( self.waiting( self.env ))
                t1 = self.env.now
                yield self.serversleeping

                # accumulate the server idle time
                self.idleTime += self.env.now - t1
            else:
                served_job_name         = 0
                served_job_arrtime      = 0
                served_job_duration     = 0

                # get the first job to be served
                if self.strat == 'SJF':
                    self.Jobs.sort( key = SJF )

                    served_job_name         = self.Jobs[0].name
                    served_job_duration     = self.Jobs[0].duration

                    j = self.Jobs.pop( 0 )
                else: # FIFO by default
                    served_job_name         = self.Jobs[0].name
                    served_job_duration     = self.Jobs[0].duration

                    j = self.Jobs.pop( 0 )

                if LOGGED:
                    if self.name == 1:
                        qlog1.write( '%.4f\t%d\t%d\n' 
                            % (self.env.now, 1 if len(self.Jobs)>0 else 0, len(self.Jobs)) )
                    else:
                        qlog2.write( '%.4f\t%d\t%d\n' 
                            % (self.env.now, 1 if len(self.Jobs)>0 else 0, len(self.Jobs)) )

                ''' sum up the waiting time'''
                self.waitingTime += self.env.now - j.arrtime
                ''' yield an event for the job finish'''
                yield self.env.timeout( j.duration )

                if self.name == 1:
                    self.nextServer.Jobs.append( Job('Job %s' %served_job_name, self.env.now, served_job_duration) )

                    ''' if server 2 is idle, wake it up'''
                    if not self.nextServer.serversleeping.triggered:
                        self.nextServer.serversleeping.interrupt( 'Wake up server, please.' )

                ''' sum up the jobs done '''
                self.jobsDone += 1

    def waiting(self, env):
        try:
            if VERBOSE:
                print( 'Server', self.name, 'is idle at %.2f' % self.env.now )
            yield self.env.timeout( MAXSIMTIME )
        except simpy.Interrupt as i:
            if VERBOSE:
                print('Server', self.name ,'waken up and works at %.2f' % self.env.now )

**The arrival process**

The arrival process is exponentially distributed which is parameterized by
1. number of servers
2. maximum number of population
3. arrival rate $\lambda$
4. service rate $\mu$
*Note that, the implementation of the arrival process embeds both arrival and service distributions.*

In [72]:
class JobGenerator:
    def __init__(self, env, server, nrjobs = 10000000, lam = 5, mu = 8):
        self.server = server

        self.nrjobs = nrjobs
        self.interarrivaltime = 1/lam;
        self.servicetime = 1/mu;
        env.process( self.generatejobs(env) )
        
    def generatejobs(self, env):
        i = 1
        while True:
            # yield an event for new job arrival
            job_interarrival = random.exponential( self.interarrivaltime )
            yield env.timeout( job_interarrival )

            # generate service time and add job to the list
            job_duration = random.exponential( self.servicetime )

            self.server.Jobs.append( Job('Job %s' %i, env.now, job_duration) )

            if VERBOSE:
                print( 'job %d: t = %.2f, l = %.2f, dt = %.2f' 
                    %( i, env.now, job_duration, job_interarrival ) )

            i += 1

            ''' if server is idle, wake it up'''
            if not self.server.serversleeping.triggered:
                self.server.serversleeping.interrupt( 'Wake up server, please.' )

**Open the log file**

If requested.

In [73]:
if LOGGED:
    qlog1 = open( 'mm1-l%d-m%d1.csv' % (LAMBDA,MU), 'w' )
    qlog1.write( '0\t0\t0\n' )
    qlog2 = open( 'mm1-l%d-m%d2.csv' % (LAMBDA,MU), 'w' )
    qlog2.write( '0\t0\t0\n' )

**Start SimPy environment**

In [74]:
env = simpy.Environment()

MyServer2 = Server( env, 2, 0, SERVICE_DISCIPLINE  )
MyServer1 = Server( env, 1, MyServer2, SERVICE_DISCIPLINE  )

MyJobGenerator = JobGenerator( env, MyServer1, POPULATION, LAMBDA, MU )

**Run the simulation** 

In [75]:
env.run( until = MAXSIMTIME )

**Close the log file**

In [76]:
if LOGGED:
    qlog1.close()
    qlog2.close()

**Print some statistics**

In [77]:
RHO = LAMBDA/MU

print('Arrivals 1               : %d' % (MyServer1.jobsDone) )
print('Arrivals 2               : %d\n' % (MyServer2.jobsDone) )


print('Idle time 1              : %d' % (MyServer1.idleTime))
print('Idle time 2              : %d\n' % (MyServer2.idleTime))


print('Waiting time 1           : %d' % (MyServer1.waitingTime))
print('Waiting time 2           : %d\n' % (MyServer2.waitingTime))


print('Job done 1               : %d' % (MyServer1.jobsDone))
print('Job done 2               : %d\n' % (MyServer2.jobsDone))


print('Utilization 1            : %.2f/%.2f' 
    % (1.0-MyServer1.idleTime/MAXSIMTIME, RHO) )
print('Utilization 2            : %.2f/%.2f\n' 
    % (1.0-MyServer2.idleTime/MAXSIMTIME, RHO) )
    

print('Mean waiting time 1      : %.2f/%.2f' 
    % (MyServer1.waitingTime/MyServer1.jobsDone, RHO**2/((1-RHO)*LAMBDA) ) )

print('Mean waiting time 2      : %.2f/%.2f' 
    % (MyServer2.waitingTime/MyServer2.jobsDone, RHO**2/((1-RHO)*LAMBDA) ) )

Arrivals 1               : 759099
Arrivals 2               : 759097

Idle time 1              : 52553
Idle time 2              : 52553

Waiting time 1           : 31381
Waiting time 2           : 40065

Job done 1               : 759099
Job done 2               : 759097

Utilization 1            : 0.47/0.47
Utilization 2            : 0.47/0.47

Mean waiting time 1      : 0.04/0.06
Mean waiting time 2      : 0.05/0.06


**Plot the statistics**

In [78]:
if LOGGED and PLOTTED:
    import matplotlib.pyplot as plt
    log = np.loadtxt( 'mm1-l%d-m%d1.csv' % (LAMBDA,MU), delimiter = '\t' )
    plt.subplot( 2, 1, 1 )
    plt.xlabel( 'Time' )
    plt.ylabel( 'Queue length' )
    plt.step( log[:200,0], log[:200,2], where='post' )
    plt.subplot( 2, 1, 2 )
    plt.xlabel( 'Time' )
    plt.ylabel( 'Server state' )
    plt.yticks([0, 1], ['idle', 'busy'])
    #plt.step( log[:200,0], log[:200,1], where='post' )
    plt.fill_between( log[:200,0], 0, log[:200,1], step="post", alpha=.4 )
    plt.tight_layout()
    plt.show()

if LOGGED and PLOTTED:
    import matplotlib.pyplot as plt
    log = np.loadtxt( 'mm1-l%d-m%d2.csv' % (LAMBDA,MU), delimiter = '\t' )
    plt.subplot( 2, 1, 1 )
    plt.xlabel( 'Time' )
    plt.ylabel( 'Queue length' )
    plt.step( log[:200,0], log[:200,2], where='post' )
    plt.subplot( 2, 1, 2 )
    plt.xlabel( 'Time' )
    plt.ylabel( 'Server state' )
    plt.yticks([0, 1], ['idle', 'busy'])
    #plt.step( log[:200,0], log[:200,1], where='post' )
    plt.fill_between( log[:200,0], 0, log[:200,1], step="post", alpha=.4 )
    plt.tight_layout()
    plt.show()